In [18]:
import requests
import pandas as pd
import json
import re

In [3]:
### PREAMBLE: LOADING LOCAL ENVIRONMENT VARIALBLES (API KEYS) Accessing secure API keys
keys_json = json.load(open("env_keys.json"))
scopus_key = keys_json['scopus_key']

In [4]:
### SCOPUS API REQUEST HEADERS
req_headers = {
    'X-ELS-ApiKey' : scopus_key
}

In [5]:
def current_download_scopus_core_len(pub_code):
    scopus_core_path = 'scopus_data/{}_scopus_core.csv'.format(pub_code.upper())
    current_scopus_df = pd.read_csv(scopus_core_path)
    obs_count = len(current_scopus_df)

    return obs_count

In [17]:
def api_core_count(pub_code):
    issn = issn_dict.get(pub_code)
    human_query_issn = 'ISSN({})'.format(issn)
    human_query_date = '{start_year_str}-{current_year_str}'.format(start_year_str=1990, current_year_str=2021)


    url = 'https://api.elsevier.com/content/search/scopus'
    call_query = {
        'httpAccept' : 'application/json',
        'query' : human_query_issn,
        'date' : human_query_date,
        'count' : '25',
        'cursor' : '*',
        'view' : 'complete'
    }

    response = requests.get(url,
                    headers = req_headers,
                    params = call_query)

    response_json = response.json()
    total_results = int(response_json['search-results']['opensearch:totalResults'])
    return total_results


In [38]:
journals_file = open('journals_of_interest.txt', 'r')
journals_files_lines = journals_file.readlines()

issn_dict = {}

real_lines = []
for line in journals_files_lines:
    if re.match(r'^-', line):
        real_lines.append(line)

for line in real_lines:
    pub_code = re.search(r'\{(.+)\}', line)[1]
    issn = re.search(r'\((.+)\)', line)[1]  
    
    issn_dict[pub_code] = issn


{'AER': '00028282',
 'ECA': '00129682',
 'JPE': '00223808',
 'QJE': '00335533',
 'RES': '00346527',
 'ATB': '0003603X',
 'ALJ': '00036056',
 'JEM': '15309134',
 'JHR': '0022166X',
 'JLE': '00222186',
 'JLO': '87566222',
 'RJE': '07416261',
 'JOF': '00221082',
 'JFE': '0304405X',
 'RFS': '08939454',
 'JOL': '0734306X'}

In [40]:
for pub_code in issn_dict.keys():
    print('-----------------------------{}-----------------------------'.format(pub_code))
    
    api_results_count = api_core_count(pub_code)
    local_results_count = current_download_scopus_core_len(pub_code)

    print('Number of expected results from Scopus Search API: {}'.format(api_results_count))
    print('Number of results downloaded locally: {}'.format(local_results_count))

    if api_results_count > local_results_count:
        print('There is something in the API missing locally. NEED TO DOWNLOAD AGAIN.')
    else:
        print('As expected. Carrying on.')
    print('-------------------------------------------------------------'.format(pub_code))


-----------------------------AER-----------------------------
Number of expected results from Scopus Search API: 5139
Number of results downloaded locally: 5139
-------------------------------------------------------------
-----------------------------ECA-----------------------------
Number of expected results from Scopus Search API: 1684
Number of results downloaded locally: 1684
-------------------------------------------------------------
-----------------------------JPE-----------------------------
Number of expected results from Scopus Search API: 1331
Number of results downloaded locally: 1331
-------------------------------------------------------------
-----------------------------QJE-----------------------------
Number of expected results from Scopus Search API: 1321
Number of results downloaded locally: 1321
-------------------------------------------------------------
-----------------------------RES-----------------------------
Number of expected results from Scopus Search 